# Downward inference


## The basics

### Model structure
Logical Neural Network correspond to a set of logical formulae which can be represented with formulae's syntax trees.

Downward inference principle will be explained on logical equality, so let us start with simple LNN which can be represented with [*logical equality*](https://en.wikipedia.org/wiki/Logical_equality) (also called bidirectional implication, since it can be expressed as two logical implications and logical and). This can be visualised as following syntax tree.

![implication_graph](images/equality_graph_plain.png)

### Downward inference principle

The downward inference is evaluation of the *operand* given the truth of the *operator* and rest of the operands.

![implication_graph](images/implication_graph.png)

The logical equality is evaluated based on its *truth table* which can be seen below.
![equality_tt](images/equality_tt.png)

So if we want to do downward inference on $P(x) \Leftrightarrow Q(x)$ we have to provide truth value of the logical equality operator and one of the operands. Lets say that whole formula is evaluated to $False$ and truth value of predicate $P(x)$ is $True$. Situation is shown on image below.

![downward_inference](images/downward_inference.png)

Now we can *infer* the truth value of predicate $Q(x)$ based on value of $P(x)$ and value of $P(x) \Leftrightarrow Q(x)$. Since our operator is logical equality we can easily use truth table above and state that value of $Q(x)$ must be $False$.

Note that we have multiple inference rules for different logical connectives. Detail can be read in [original paper](https://arxiv.org/pdf/2006.13155).

We can apply downward inference to multiple values of our predicates:

![downward_inference](images/predicate_down.png)


## Grecian domain

Now we are ready to move to our specific example and finally create a LNN model. We will continue with our grecian domain.

We will extend it by new statement based on [ontology for word "grecian"](http://wordnetweb.princeton.edu/perl/webwn?o2=&o0=1&o8=1&o1=1&o7=&o5=&o9=&o6=&o3=&o4=&s=grecian&i=0&h=00#c). Let us modify description of our domain in natural language:

* All grecian is a native or resident of Greece.<br>
* Grecians are mortal.<br>
* Zeus is resident of Greece (is not native of Greece), is not mortal.<br>
* Socrates is native of Greece (is not resident of Greece), is mortal.<br>
* Confucius is neither native or resident of Greece, is mortal.<br>
* Great Pyramid of Giza is neither native or resident of Greece, is not mortal.

### First order logic description
Now let us translate this domain into First-order logic.

#### Predicates
Since we extended our example domain we have to verify if we define all needed predicates. In fact we need two new predicates $native\_of\_greece(x)$ and $resident\_of\_greece(x)$ to capture the relations like "Zeus is resident of Greece".

Rest of the predicate $grecian(x)$ and $mortal(x)$ stays same.

#### Formulae
The translation of "All grecian is a native or resident of Greece" sentence is little bit more complex, let us take it one by one. We should be able to see two previously defined predicates $native\_of\_greece(x)$ and $resident\_of\_greece(x)$.

Since "native or resident of Greece" includes word "or" we have to introduce [*logical disjunction*](https://en.wikipedia.org/wiki/Logical_disjunction). Logical disjunction is $False$ when both of the operands are $False$.

So we have:

$native\_of\_greece(x) \lor resident\_of\_greece(x)$

We can continue to next part of the natural language sentence "grecian is a native or resident of Greece". This part includes our previously defined predicate $grecian(x)$ and word "is". The word "is" can be translated as logical equality which was previously defined.

Let us extend our formula:

$grecian(x) \Leftrightarrow native\_of\_greece(x) \lor resident\_of\_greece(x)$

The word "all" represent so called [*universal quantificator*](https://en.wikipedia.org/wiki/Universal_quantification) $\forall x$ to state that for all objects $x$, given formula must be evaluated to $True$, otherwise the whole quantified formula will be evaluated to $False$.

$(\forall x) (grecian(x) \Leftrightarrow native\_of\_greece(x) \lor resident\_of\_greece(x))$

Moreover we will say that this formula will be an [axiom](https://en.wikipedia.org/wiki/Axiom) which means that it can be taken as true statement in our domain in other words its satisfied by definition on our domain. This part will be crucial for downward inference.

#### Objects
Our natural language description include same set of objects as in previous example. Let's state it for completeness `{Zeus, Socrates, Confucius, Great Pyramid of Giza}`. Their relations are represented by following table:

![grecian_downward_tt](images/grecian_downward_tt_predicates.png)

### Our goal
Since we do not have truth value for $grecian(x)$ predicate in our domain description, we would like to infer truth values for $grecian(x)$ predicate for each $x$ (object) in our domain.

![grecian_downward_tt](images/grecian_downward_tt.png)

### Definition of LNN model

Now we will define our model in LNN library.

First we will create an empty model.

In [2]:
from lnn import Model

model = Model()

Define our predicates and add them to the model.

In [3]:
from lnn import Predicate

grecian = Predicate(name="grecian")
mortal = Predicate(name="mortal")
native_of_greece = Predicate(name="native_of_greece")
resident_of_greece = Predicate(name="resident_of_greece")

model.add_formulae(grecian, mortal, native_of_greece, resident_of_greece)

Now we create one variable which will be used in definitions of other predicates. Node that we use parameter `world=World.AXIOM` in `ForAll` formula to state that formula is an axiom.

In [4]:
from lnn import Variable, Implies, World, ForAll, Or, Bidirectional

x = Variable("x")

all_grecians_are_native_or_resident = ForAll(
    x,
    Bidirectional(
        Or(native_of_greece(x), resident_of_greece(x), name="native_or_resident"),
        grecian(x),
        name="grecians_are_native_or_resident",
    ),
    name="all_grecians_are_native_or_resident",
    world=World.AXIOM,
)

grecians_are_mortals = Implies(grecian(x), mortal(x), name="grecians_are_mortals")

model.add_formulae(grecians_are_mortals, all_grecians_are_native_or_resident)


Add facts provided by domain descripton.

In [5]:
from lnn import Fact

model.add_facts(
    {
        "native_of_greece": {
            "Great Pyramid of Giza": Fact.FALSE,
            "Confucius": Fact.FALSE,
            "Zeus": Fact.FALSE,
            "Socrates": Fact.TRUE,
        },
        "resident_of_greece": {
            "Great Pyramid of Giza": Fact.FALSE,
            "Confucius": Fact.FALSE,
            "Zeus": Fact.TRUE,
            "Socrates": Fact.FALSE,
        },
        "mortal": {
            "Great Pyramid of Giza": Fact.FALSE,
            "Confucius": Fact.TRUE,
            "Zeus": Fact.FALSE,
            "Socrates": Fact.TRUE,
        },
    }
)

### Downward inference inside LNN model

Since downward inference is evaluation of the operand given the truth of the operator and rest of the operands we first do upward inference to evaluate maximum number of formulae in our model. These result can be later used in downward inference to infer truth value for our facts.

In [6]:
from lnn import Direction

model.infer(direction=Direction.UPWARD)

(1, tensor(10., grad_fn=<AddBackward0>))

Now we can verify that upward inference is not enough to infer facts of our $grecian(x)$ predicate.

In [7]:
model["grecian"].print()

OPEN   Predicate: grecian(x0) 
'Great Pyramid of Giza'                                  UNKNOWN (0.0, 1.0)
'Confucius'                                              UNKNOWN (0.0, 1.0)
'Zeus'                                                   UNKNOWN (0.0, 1.0)
'Socrates'                                               UNKNOWN (0.0, 1.0)



To infer it, we have to start downward inference which will use truth values of formulae which were calculated in upward inference to infer truth values of our facts and mainly our axiom which is true by definition.

In [8]:
model.infer(direction=Direction.DOWNWARD)

(1, tensor(12., grad_fn=<AddBackward0>))

And as we can see, model correctly infered truth value for $grecian(x)$ predicate.

In [9]:
model["grecian"].print()

OPEN   Predicate: grecian(x0) 
'Great Pyramid of Giza'                                    FALSE (0.0, 0.0)
'Confucius'                                                FALSE (0.0, 0.0)
'Zeus'                                                      TRUE (1.0, 1.0)
'Socrates'                                                  TRUE (1.0, 1.0)



How was that possible? Remember the example from the beginning. We already know the truth value for formula with logical disjuntion (or):

In [10]:
model["native_or_resident"].print()

OPEN   Or: native_or_resident(x) 
'Great Pyramid of Giza'                                    FALSE (0.0, 0.0)
'Confucius'                                                FALSE (0.0, 0.0)
'Zeus'                                                      TRUE (1.0, 1.0)
'Socrates'                                                  TRUE (1.0, 1.0)



Since we know that $(\forall x) (native\_of\_greece(x) \lor resident\_of\_greece(x)) \Leftrightarrow grecian(x)$ is an axiom (so we know the truth value for each $x$) and truth values of first operand $(native\_of\_greece(x) \lor resident\_of\_greece(x))$ are infered by upward inference, we can then infer the truth value of $grecian(x)$ as we did in the beginning.